# SQL Practice 12/1/2022

## Q1

Given the table of transactions below, write a query to obtain the third transaction of every user. Output the user_id, spend, and transaction_date.

Data Information

`transactions` **Table**

Column Name | Type
------------|-----
user_id | integer
spend | decimal
transaction_date | timestamp

My solution is below

```SQL
WITH ranked_table AS (
  SELECT *,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY transaction_date) as ranking
  FROM transactions
)

SELECT user_id, spend, transaction_date
FROM ranked_table
WHERE ranking = 3

;
```

So we first add a new column of row numbers partitioned by the user_id to our table as a CTE. We order this row numbering by the transaction date in ascending order. Then in the actual `SELECT` clause we filter by the which row is numbered third, and since they are ordered by transaction date this will be the users third ever transaction.

## Q2

You are given a table containing information on users, their ages, and their time spent on a specific activity. Write a query to obtain a breakdown of the time spent sending vs opening as a percentage of the total time spent on these activities for each age group. Your output should be the age bucket and the percentage of sendig and opening. Round the percentages to 2 decimal places.

Data Information

`activities` **Table**

Column Name | Type
------------|------
activity_id | integer
user_id | integer
activity_type | string
time_spent | float
activity_date | datetime

`age_breakdown` **Table**

Column Name | Type
------------|------
user_id | integer
age_bucket | string

The `activity_type` can only be one of three options `"send"`, `"open"`, or `"chat"`.
The `age_bucket` can only be one of three options `"21-25"`, `"26-30"`, or `"31-35"`

My solution is below

```SQL
WITH joined_table AS (
  SELECT activities.activity_id,
    activities.user_id,
    age_breakdown.age_bucket,
    activities.activity_type,
    activities.time_spent
  FROM activities LEFT JOIN age_breakdown
    ON activities.user_id = age_breakdown.user_id
  
  
)

SELECT age_bucket,
  ROUND(
    100.0 * 
    SUM(CASE WHEN activity_type = 'send' THEN time_spent ELSE NULL END) / 
    SUM(CASE WHEN activity_type <> 'chat' THEN time_spent ELSE NULL END)
    , 2) AS send_perc,
  ROUND(
    100.0 * 
    SUM(CASE WHEN activity_type = 'open' THEN time_spent ELSE NULL END) / 
    SUM(CASE WHEN activity_type <> 'chat' THEN time_spent ELSE NULL END)
    , 2) AS open_perc  
FROM joined_table

GROUP BY age_bucket

;
```

So here we are first joining the table and making sure to only import the columns that we need to create the desired output. Then we compute the percentages using a `CASE` command. However, I originally defaulted to the following logic:

```SQL
 ROUND(
    100.0 * 
    SUM(CASE WHEN activity_type = 'send' THEN 1 ELSE NULL END) / 
    SUM(CASE WHEN activity_type <> 'chat' THEN 1 ELSE NULL END)
    , 2)
```

This is incorrect as it computes the percentage of activities that are "send" activities out of the total number of send and open activities, but it does not compute the percentage of time spent!

```SQL

```